In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

In [2]:
modelo=pd.read_csv('data/modelo.csv',low_memory=False)

In [3]:
modelo.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
modelo.isnull().sum()

timestamp                         0
event                             0
person                            0
url                         2150550
sku                         1021151
model                       1020168
condition                   1021151
storage                     1021151
color                       1021151
skus                        1835732
search_term                 2227918
staticpage                  2330480
campaign_source             2150395
search_engine               2235275
channel                     2137612
new_vs_returning            2137612
city                        2137612
region                      2137612
country                     2137612
device_type                 2137612
screen_resolution           2137615
operating_system_version    2137612
browser_version             2137612
dtype: int64

In [5]:
modelo.drop(['url', 'sku', 'skus','channel','staticpage','condition','storage','color','search_engine','device_type','campaign_source','country','region','search_term','new_vs_returning','city','region','screen_resolution','operating_system_version','browser_version'], axis=1, inplace=True)

In [6]:
training= pd.read_csv('data/training.csv',low_memory=False)

# Armado del modelo

In [7]:
modelo.sort_values('person',inplace=True)

In [8]:
import calendar

modelo['timestamp'] = pd.to_datetime(modelo['timestamp'])
modelo['dia']=modelo['timestamp'].dt.dayofyear


modelo['dias_distintos'] = modelo['person'].map(modelo.groupby(['person',(modelo['timestamp'].dt.month==5) ] )['dia'].nunique().unstack()[True])   #ultimo mes
modelo['dias_distintos']=modelo['dias_distintos'].fillna(0).astype(int)

modelo['ultima_visita']=modelo['person'].map(modelo.groupby('person' )['dia'].max())    
modelo['ultima_visita']=modelo['ultima_visita'].fillna(0).astype(int)
modelo.drop('dia',1,inplace=True)


modelo['modelos_distintos_vistos']= modelo['person'].map( modelo.groupby('person')['model'].nunique())
modelo.drop('model',1,inplace=True)


modelo['campaña_ultima_semana']=modelo['person'].map(modelo.groupby(['person',(modelo['timestamp'].dt.month==5) & (modelo['timestamp'].dt.day>23) & (modelo['event']=="ad campaign hit")]).size().unstack()[True]) 
modelo['campaña_ultima_semana']=modelo['campaña_ultima_semana'].fillna(0).astype(int)


modelo['busqueda_celular']=modelo['person'].map(modelo.groupby(['person',(modelo['timestamp'].dt.month==5) & (modelo['timestamp'].dt.day>23) & (modelo['event']=="searched products")]).size().unstack()[True]) 
modelo['busqueda_celular']=modelo['busqueda_celular'].fillna(0).astype(int)


modelo['total_visitas_usuario'] = modelo.groupby('person')['person'].transform('count')


modelo['visitas_ultimo_mes']=modelo['person'].map(modelo.groupby(['person',(modelo['timestamp'].dt.month)==5]).size().unstack()[True])
modelo['visitas_ultimo_mes']=modelo['visitas_ultimo_mes'].fillna(0).astype(int)


modelo['visitas_ultimos_15']=modelo['person'].map(modelo.groupby(['person', (modelo['timestamp'].dt.month==5)& (modelo['timestamp'].dt.day>15)]).size().unstack()[True])
modelo['visitas_ultimos_15']=modelo['visitas_ultimos_15'].fillna(0).astype(int)


modelo['visitas_ultima_semana']=modelo['person'].map(modelo.groupby(['person', (modelo['timestamp'].dt.month==5)& (modelo['timestamp'].dt.day>23)]).size().unstack()[True])
modelo['visitas_ultima_semana']=modelo['visitas_ultima_semana'].fillna(0).astype(int)


modelo['visitas_ultimos_3']=modelo['person'].map(modelo.groupby(['person', (modelo['timestamp'].dt.month==5)& (modelo['timestamp'].dt.day>27)]).size().unstack()[True])
modelo['visitas_ultimos_3']=modelo['visitas_ultimos_3'].fillna(0).astype(int)


modelo['total_checkout']=modelo['person'].map(modelo.groupby(['person','event']).size().unstack()['checkout'])
modelo['total_checkout']=modelo['total_checkout'].fillna(0).astype(int)


modelo['checkouts_ultimo_mes']=modelo['person'].map(modelo.groupby(['person','event',modelo['timestamp'].dt.month==5 ]).size().unstack()[True].unstack()['checkout'])   
modelo['checkouts_ultimo_mes']=modelo['checkouts_ultimo_mes'].fillna(0).astype(int)



modelo['checkouts_ultima_semana']=modelo['person'].map(modelo.groupby(['person','event',(modelo['timestamp'].dt.month==5)& (modelo['timestamp'].dt.day>23)]).size().unstack()[True].unstack()['checkout'])
modelo['checkouts_ultima_semana']=modelo['checkouts_ultima_semana'].fillna(0).astype(int)



modelo['total_compras']=modelo['person'].map(modelo.groupby(['person','event']).size().unstack()['conversion'])
modelo['total_compras']=modelo['total_compras'].fillna(0).astype(int)



modelo['compras_ultimo_mes']=modelo['person'].map(modelo.groupby(['person','event',modelo['timestamp'].dt.month==5 ]).size().unstack()[True].unstack()['conversion'])
modelo['compras_ultimo_mes']=modelo['compras_ultimo_mes'].fillna(0).astype(int)



modelo['compras_ultima_semana']=modelo['person'].map(modelo.groupby(['person','event',(modelo['timestamp'].dt.month==5)& (modelo['timestamp'].dt.day>23)]).size().unstack()[True].unstack()['conversion'])
modelo['compras_ultima_semana']=modelo['compras_ultima_semana'].fillna(0).astype(int)


modelo['subscripciones_ultimo_mes']=modelo['person'].map(modelo.groupby(['person','event',modelo['timestamp'].dt.month==5 ]).size().unstack()[True].unstack()['lead'])
modelo['subscripciones_ultimo_mes']=modelo['subscripciones_ultimo_mes'].fillna(0).astype(int)



In [9]:
modelo=modelo.drop_duplicates(subset=['person'])
modelo.drop(['timestamp','event'], 1, inplace=True)

In [10]:
modelo=pd.merge(modelo, training, on='person', how='left')

In [11]:
modelo.head()

,person,dias_distintos,ultima_visita,modelos_distintos_vistos,campaña_ultima_semana,busqueda_celular,total_visitas_usuario,visitas_ultimo_mes,visitas_ultimos_15,visitas_ultima_semana,visitas_ultimos_3,total_checkout,checkouts_ultimo_mes,checkouts_ultima_semana,total_compras,compras_ultimo_mes,compras_ultima_semana,subscripciones_ultimo_mes,label
0,0008ed71,1,137,3,0,0,6,6,6,0,0,3,3,0,0,0,0,0,0.0
1,00091926,22,151,36,6,0,448,448,261,47,19,2,2,1,0,0,0,0,NaN
2,00091a7a,0,85,3,0,0,10,0,0,0,0,0,0,0,0,0,0,0,NaN
3,000ba417,3,146,26,0,0,206,206,206,138,0,6,6,3,1,1,1,0,NaN
4,000c79fe,1,149,1,1,9,17,17,17,17,17,1,1,1,0,0,0,0,0.0


In [12]:
a_predecir=modelo[modelo['label'].isnull()]
a_predecir.head()

,person,dias_distintos,ultima_visita,modelos_distintos_vistos,campaña_ultima_semana,busqueda_celular,total_visitas_usuario,visitas_ultimo_mes,visitas_ultimos_15,visitas_ultima_semana,visitas_ultimos_3,total_checkout,checkouts_ultimo_mes,checkouts_ultima_semana,total_compras,compras_ultimo_mes,compras_ultima_semana,subscripciones_ultimo_mes,label
1,00091926,22,151,36,6,0,448,448,261,47,19,2,2,1,0,0,0,0,NaN
2,00091a7a,0,85,3,0,0,10,0,0,0,0,0,0,0,0,0,0,0,NaN
3,000ba417,3,146,26,0,0,206,206,206,138,0,6,6,3,1,1,1,0,NaN
5,000e4d9e,9,150,37,0,0,411,411,411,112,75,1,1,0,0,0,0,0,NaN
6,000e619d,4,138,23,0,0,68,68,58,0,0,1,1,0,0,0,0,0,NaN


In [13]:
modelo.drop(modelo[modelo['label'].isnull()].index,inplace=True)
modelo['label']=modelo['label'].astype(int)
modelo.head()

,person,dias_distintos,ultima_visita,modelos_distintos_vistos,campaña_ultima_semana,busqueda_celular,total_visitas_usuario,visitas_ultimo_mes,visitas_ultimos_15,visitas_ultima_semana,visitas_ultimos_3,total_checkout,checkouts_ultimo_mes,checkouts_ultima_semana,total_compras,compras_ultimo_mes,compras_ultima_semana,subscripciones_ultimo_mes,label
0,0008ed71,1,137,3,0,0,6,6,6,0,0,3,3,0,0,0,0,0,0
4,000c79fe,1,149,1,1,9,17,17,17,17,17,1,1,1,0,0,0,0,0
10,001802e4,1,151,2,5,4,19,19,19,19,19,1,1,1,0,0,0,0,0
12,0019e639,9,142,26,0,0,471,290,136,0,0,15,4,0,2,1,0,0,0
14,001b0bf9,1,142,1,0,0,7,7,7,0,0,2,2,0,1,1,0,0,0


In [14]:
len(modelo[modelo['label']==1]),len(modelo[modelo['label']==0]) 

(980, 18434)

In [15]:
len(modelo[modelo['label']==1])*100/len(modelo['label']),len(modelo[modelo['label']==0])*100/len(modelo['label'])

(5.047903574739879, 94.95209642526012)

In [16]:
modelo[modelo['label']==1].head()

,person,dias_distintos,ultima_visita,modelos_distintos_vistos,campaña_ultima_semana,busqueda_celular,total_visitas_usuario,visitas_ultimo_mes,visitas_ultimos_15,visitas_ultima_semana,visitas_ultimos_3,total_checkout,checkouts_ultimo_mes,checkouts_ultima_semana,total_compras,compras_ultimo_mes,compras_ultima_semana,subscripciones_ultimo_mes,label
31,003476c0,1,132,2,0,0,43,14,0,0,0,0,0,0,0,0,0,0,1
38,003d4cac,0,116,2,0,0,8,0,0,0,0,1,0,0,0,0,0,0,1
39,003dd965,1,136,0,0,0,5,5,5,0,0,0,0,0,0,0,0,0,1
43,0042421c,0,100,5,0,0,22,0,0,0,0,0,0,0,0,0,0,0,1
57,006671fd,0,118,5,0,0,18,0,0,0,0,0,0,0,0,0,0,0,1


In [17]:
modelo[modelo['label']==0].head()

,person,dias_distintos,ultima_visita,modelos_distintos_vistos,campaña_ultima_semana,busqueda_celular,total_visitas_usuario,visitas_ultimo_mes,visitas_ultimos_15,visitas_ultima_semana,visitas_ultimos_3,total_checkout,checkouts_ultimo_mes,checkouts_ultima_semana,total_compras,compras_ultimo_mes,compras_ultima_semana,subscripciones_ultimo_mes,label
0,0008ed71,1,137,3,0,0,6,6,6,0,0,3,3,0,0,0,0,0,0
4,000c79fe,1,149,1,1,9,17,17,17,17,17,1,1,1,0,0,0,0,0
10,001802e4,1,151,2,5,4,19,19,19,19,19,1,1,1,0,0,0,0,0
12,0019e639,9,142,26,0,0,471,290,136,0,0,15,4,0,2,1,0,0,0
14,001b0bf9,1,142,1,0,0,7,7,7,0,0,2,2,0,1,1,0,0,0


In [18]:
modelo[modelo['label']==1].mean()-modelo[modelo['label']==0].mean()

dias_distintos                0.600485
ultima_visita                -6.780611
modelos_distintos_vistos      3.446072
campaña_ultima_semana         0.343258
busqueda_celular              0.586515
total_visitas_usuario        43.553997
visitas_ultimo_mes           16.661781
visitas_ultimos_15            9.752915
visitas_ultima_semana        10.056608
visitas_ultimos_3             8.237521
total_checkout                0.165994
checkouts_ultimo_mes         -0.411831
checkouts_ultima_semana      -0.147231
total_compras                 0.275212
compras_ultimo_mes            0.119354
compras_ultima_semana         0.080599
subscripciones_ultimo_mes     0.006040
label                         1.000000
dtype: float64

## MACHINE LEARNING

In [19]:
##Ahora empieza ML

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score,precision_score,recall_score,roc_auc_score
from sklearn.model_selection import GridSearchCV,StratifiedKFold,RandomizedSearchCV

## Split

In [20]:
X = modelo.drop(['person','label'], axis=1)
y = modelo['label']

In [21]:
from sklearn.feature_selection import SelectKBest
model= SelectKBest(k=12).fit(X,y)
columnas=list(X.columns[model.get_support()])
columnas

['dias_distintos',
 'ultima_visita',
 'modelos_distintos_vistos',
 'total_visitas_usuario',
 'visitas_ultimo_mes',
 'visitas_ultimos_15',
 'visitas_ultima_semana',
 'visitas_ultimos_3',
 'checkouts_ultimo_mes',
 'total_compras',
 'compras_ultimo_mes',
 'compras_ultima_semana']

In [22]:
model=model.transform(X)

In [23]:
X = pd.DataFrame.from_records(model)
X.columns=columnas

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=42,test_size=0.2) 

In [25]:
len(X_train),len(X_test),len(y_train),len(y_test)

(15531, 3883, 15531, 3883)

In [26]:
X_train.head()

,dias_distintos,ultima_visita,modelos_distintos_vistos,total_visitas_usuario,visitas_ultimo_mes,visitas_ultimos_15,visitas_ultima_semana,visitas_ultimos_3,checkouts_ultimo_mes,total_compras,compras_ultimo_mes,compras_ultima_semana
12637,2,137,6,36,36,36,0,0,2,0,0,0
18143,3,150,16,225,225,225,225,225,8,0,0,0
11430,2,148,9,44,44,20,20,20,2,0,0,0
15861,1,142,1,3,3,3,0,0,1,0,0,0
10505,3,146,8,29,29,29,8,0,0,0,0,0


In [27]:
y_train.head()

25350    0
36328    0
22956    0
31787    0
21032    1
Name: label, dtype: int32

## Algoritmos de clasificación

## XGBOOST

In [28]:
import xgboost as xgb
my_classifier1 = xgb.XGBClassifier(objective ='binary:logistic',max_depth =1,learning_rate = 0.02, n_estimators = 500,scale_pos_weight=8)
my_classifier1.fit(X_train,y_train)
predictions=my_classifier1.predict(X_test)
metrics.confusion_matrix(y_test, predictions)

array([[3154,  518],
       [  71,  140]], dtype=int64)

In [29]:
accuracy_score(y_test, predictions),roc_auc_score(y_test,  my_classifier1.predict_proba(X_test)[:,1])

(0.8483131599278908, 0.8721443948827556)

## RANDOM FOREST

In [30]:
from sklearn.ensemble import RandomForestClassifier #a mayor depth, mas precisos los 0, menos los 1,class weight cambia mucho
my_classifier2 = RandomForestClassifier(criterion= 'entropy',max_depth= 5,n_estimators=1000, max_features= 'auto', class_weight={1:9} ,n_jobs=-1)
my_classifier2.fit(X_train,y_train)
predictions=my_classifier2.predict(X_test)
metrics.confusion_matrix(y_test, predictions)

array([[3175,  497],
       [  78,  133]], dtype=int64)

In [31]:
accuracy_score(y_test, predictions),roc_auc_score(y_test,  my_classifier2.predict_proba(X_test)[:,1])

(0.8519186196240021, 0.8690764747183759)

## Voting classifier

In [32]:
from sklearn.ensemble import VotingClassifier
vclf = VotingClassifier(estimators=[  ('XGB', my_classifier1), ('RF', my_classifier2)], voting='soft', n_jobs =-1)
vclf = vclf.fit(X_train, y_train)
predictions = vclf.predict(X_test)
metrics.confusion_matrix(y_test, predictions)

array([[3167,  505],
       [  73,  138]], dtype=int64)

In [33]:
accuracy_score(y_test, predictions),roc_auc_score(y_test,  vclf.predict_proba(X_test)[:,1])

(0.8511460211176926, 0.8731046526035374)

## Random search

In [34]:
"""
vclf = VotingClassifier(estimators=[  ('XGB', xgb.XGBClassifier(objective ='binary:logistic' , max_depth =1,learning_rate = 0.01, n_estimators = 400 )),
                                    ('RF', RandomForestClassifier(n_jobs=-1,max_features= 'auto',  max_depth= 5,n_estimators=1000  ))], voting='soft', n_jobs =-1)

params = {'XGB__scale_pos_weight': [i for i in range(1,10)],
        'RF__class_weight': [  {1: i for i in range(1,10)}],
        'RF__criterion': ['gini','entropy' ], 
         }

folds = 10
param_comb = 20

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(vclf, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=-1, cv=skf.split(X_train,y_train), verbose=3, random_state=1001 )
random_search.fit(X_train, y_train)
print('\n Best hyperparameters:')
print(random_search.best_params_)
resultados=pd.DataFrame(random_search.cv_results_)
resultados=resultados.sort_values('mean_test_score',ascending=False)
resultados.head()
"""

"\nvclf = VotingClassifier(estimators=[  ('XGB', xgb.XGBClassifier(objective ='binary:logistic' , max_depth =1,learning_rate = 0.01, n_estimators = 400 )),\n                                    ('RF', RandomForestClassifier(n_jobs=-1,max_features= 'auto',  max_depth= 5,n_estimators=1000  ))], voting='soft', n_jobs =-1)\n\nparams = {'XGB__scale_pos_weight': [i for i in range(1,10)],\n        'RF__class_weight': [  {1: i for i in range(1,10)}],\n        'RF__criterion': ['gini','entropy' ], \n         }\n\nfolds = 10\nparam_comb = 20\n\nskf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)\n\nrandom_search = RandomizedSearchCV(vclf, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=-1, cv=skf.split(X_train,y_train), verbose=3, random_state=1001 )\nrandom_search.fit(X_train, y_train)\nprint('\n Best hyperparameters:')\nprint(random_search.best_params_)\nresultados=pd.DataFrame(random_search.cv_results_)\nresultados=resultados.sort_values('mean_te

## KAGGLE PREDICTIONS

In [35]:
kaggle= pd.read_csv('data/kaggle.csv',low_memory=False)
kaggle=pd.merge(kaggle, a_predecir, on='person', how='inner')

In [36]:
a_predecir.head()

,person,dias_distintos,ultima_visita,modelos_distintos_vistos,campaña_ultima_semana,busqueda_celular,total_visitas_usuario,visitas_ultimo_mes,visitas_ultimos_15,visitas_ultima_semana,visitas_ultimos_3,total_checkout,checkouts_ultimo_mes,checkouts_ultima_semana,total_compras,compras_ultimo_mes,compras_ultima_semana,subscripciones_ultimo_mes,label
1,00091926,22,151,36,6,0,448,448,261,47,19,2,2,1,0,0,0,0,NaN
2,00091a7a,0,85,3,0,0,10,0,0,0,0,0,0,0,0,0,0,0,NaN
3,000ba417,3,146,26,0,0,206,206,206,138,0,6,6,3,1,1,1,0,NaN
5,000e4d9e,9,150,37,0,0,411,411,411,112,75,1,1,0,0,0,0,0,NaN
6,000e619d,4,138,23,0,0,68,68,58,0,0,1,1,0,0,0,0,0,NaN


In [37]:
kaggle2=kaggle.loc[:,columnas]
kaggle2.head()

,dias_distintos,ultima_visita,modelos_distintos_vistos,total_visitas_usuario,visitas_ultimo_mes,visitas_ultimos_15,visitas_ultima_semana,visitas_ultimos_3,checkouts_ultimo_mes,total_compras,compras_ultimo_mes,compras_ultima_semana
0,1,138,1,9,9,9,0,0,1,0,0,0
1,15,148,15,567,173,126,53,8,2,0,0,0
2,2,147,3,26,26,26,16,0,1,0,0,0
3,13,151,57,836,836,449,270,252,1,0,0,0
4,5,142,9,257,257,257,0,0,2,0,0,0


In [38]:
predictions=vclf.predict_proba(kaggle2)

In [39]:
kaggle['prediction0'] = predictions[:,0]
kaggle['label'] = predictions[:,1]

In [40]:
kaggle.head()

,person,dias_distintos,ultima_visita,modelos_distintos_vistos,campaña_ultima_semana,busqueda_celular,total_visitas_usuario,visitas_ultimo_mes,visitas_ultimos_15,visitas_ultima_semana,visitas_ultimos_3,total_checkout,checkouts_ultimo_mes,checkouts_ultima_semana,total_compras,compras_ultimo_mes,compras_ultima_semana,subscripciones_ultimo_mes,label,prediction0
0,4886f805,1,138,1,0,0,9,9,9,0,0,1,1,0,0,0,0,0,0.058919,0.941081
1,0297fc1e,15,148,15,1,0,567,173,126,53,8,7,2,0,0,0,0,0,0.275909,0.724091
2,2d681dd8,2,147,3,1,1,26,26,26,16,0,1,1,1,0,0,0,0,0.094565,0.905435
3,cccea85e,13,151,57,5,1,836,836,449,270,252,1,1,0,0,0,0,0,0.335232,0.664768
4,4c8a8b93,5,142,9,0,0,257,257,257,0,0,2,2,0,0,0,0,0,0.211328,0.788672


In [41]:
kaggle[['person','label']].to_csv('predicciones3.csv', encoding='utf-8', index=False)